<a href="https://colab.research.google.com/github/Tar-ive/STOCKS/blob/code/Trading_Stratergies_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright © [2024] [Saksham Adhikari]
All rights reserved.

In [1]:
!pip install yfinance

In [2]:
!pip install quantstats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 740.3 kB/s eta 0:00:00


In [3]:
!pip install pyportfolioopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 965.7 kB/s eta 0:00:00


In [ ]:
!pip install ta


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=1d0b5bc4ab52713ce33ad72cd539c27b8223a9535ba2269fbcfcc9763384af6a
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
end_date = dt.datetime.now() # Defining the datetime for March 21st
start_date = end_date - dt.timedelta(days=729) # Loading hourly data for the last 729 days

hourly_nvda = yf.download('NVDA', start=start_date, end=end_date, interval='1h')
hourly_nvda

In [ ]:
hourly_nvda['rsi'] = ta.momentum.RSIIndicator(hourly_nvda['Adj Close'], window = 14).rsi()
hourly_nvda

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,row_heights=[0.7, 0.3])
fig.add_trace(go.Candlestick(x=hourly_nvda.index,
                             open=hourly_nvda['Open'],
                             high=hourly_nvda['High'],
                             low=hourly_nvda['Low'],
                             close=hourly_nvda['Adj Close'],
                             name='NVDA'), row=1, col=1)

# Adding annotation
fig.add_annotation(text='NVDA 1HR',
                    font=dict(color='white', size=40),
                    xref='paper', yref='paper',
                    x=0.5, y=0.65,
                    showarrow=False,
                    opacity=0.2)

fig.add_trace(go.Scatter(x=hourly_nvda.index,
                         y=hourly_nvda['rsi'],
                         mode='lines',
                         line=dict(color='#CBC3E3'),
                         name='RSI'),
              row=2, col=1)


# Adding marking lines at 70 and 30 levels
fig.add_shape(type="line",
              x0=hourly_nvda.index[0], y0=70, x1=hourly_nvda.index[-1], y1=70,
              line=dict(color="red", width=2, dash="dot"),
              row=2, col=1)
fig.add_shape(type="line",
              x0=hourly_nvda.index[0], y0=30, x1=hourly_nvda.index[-1], y1=30,
              line=dict(color="#90EE90", width=2, dash="dot"),
              row=2, col=1)

# Layout
fig.update_layout(title='NVDA Hourly Candlestick Chart from 2022 to 2024',
                  yaxis=dict(title='Price'),
                  height=1000,
                 template = 'plotly_dark')

# Configuring subplots
fig.update_xaxes(rangeslider_visible=False)
fig.update_yaxes(title_text='Price', row = 1)
fig.update_yaxes(title_text='RSI', row = 2)

fig.show()
fig.write_html("nvda_hourly1.html")

In [ ]:
# Defining the parameters for the RSI strategy
rsi_period = 14
overbought = 70
oversold = 30

# Creating a new column to hold the signals
hourly_nvda['signal'] = 0 # When we're not positioned, 'signal' = 0

# Generating the entries
for i in range(rsi_period, len(hourly_nvda)):
    if hourly_nvda['rsi'][i] > overbought and hourly_nvda['rsi'][i - 1] <= overbought:
        hourly_nvda['signal'][i] = -1 # We sell when 'signal' = -1
    elif hourly_nvda['rsi'][i] < oversold and hourly_nvda['rsi'][i - 1] >= oversold:
        hourly_nvda['signal'][i] = 1 # We buy when 'signal' = 1

# Calculating the pair's daily returns
hourly_nvda['returns'] = hourly_nvda['Adj Close'].pct_change()
hourly_nvda['cumulative_returns'] = (1 + hourly_nvda['returns']).cumprod() - 1 # Total returns of the period

# Applying the signals to the returns
hourly_nvda['strategy_returns'] = hourly_nvda['signal'].shift(1) * hourly_nvda['returns']

# Calculating the cumulative returns of the strategy
hourly_nvda['cumulative_strategy_returns'] = (1 + hourly_nvda['strategy_returns']).cumprod() - 1

# Setting the initial capital to $100
initial_capital = 100

# Calculating total portfolio value
hourly_nvda['portfolio_value'] = (1 + hourly_nvda['strategy_returns']).cumprod() * initial_capital

# Printing the number of trades, initial capital, and final capital
num_trades = hourly_nvda['signal'].abs().sum()
final_capital = hourly_nvda['portfolio_value'].iloc[-1]
total_return = (final_capital - initial_capital) / initial_capital * 100

print('\n')
print(f"Number of trades: {num_trades}")
print(f"Initial capital: ${initial_capital}")
print(f"Final capital: ${final_capital:.2f}")
print(f"Total return: {total_return:.2f}%")
print('\n')

# Plotting the portfolio total value
fig = go.Figure()

fig.add_trace(go.Scatter(x=hourly_nvda.index,
                         y=hourly_nvda['portfolio_value'].round(2),
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='Portfolio Value'))

fig.update_layout(title='Portfolio - RSI Strategy on Hourly Data',
                  xaxis_title='Date',
                  yaxis_title='Value ($)',
                  template='plotly_dark',
                 height = 600)

fig.show()
fig.write_html("nvda_rsi.html")

In [ ]:
#MACD Strategy
hourly_nvda['ema12'] = hourly_nvda['Adj Close'].ewm(span=12, adjust=False).mean()
hourly_nvda['ema26'] = hourly_nvda['Adj Close'].ewm(span=26, adjust=False).mean()
hourly_nvda['macd'] = hourly_nvda['ema12'] - hourly_nvda['ema26']
# Signal Line = 9-period EMA of MACD Line
hourly_nvda['signal_line'] = hourly_nvda['macd'].ewm(span=9, adjust=False).mean()

# Create a column for the trading signal
hourly_nvda['signal'] = 0
# Buy signal (MACD crosses above Signal Line)
hourly_nvda.loc[(hourly_nvda['macd'] > hourly_nvda['signal_line']) &
                (hourly_nvda['macd'].shift(1) <= hourly_nvda['signal_line'].shift(1)), 'signal'] = 1
# Sell signal (MACD crosses below Signal Line)
hourly_nvda.loc[(hourly_nvda['macd'] < hourly_nvda['signal_line']) &
                (hourly_nvda['macd'].shift(1) >= hourly_nvda['signal_line'].shift(1)), 'signal'] = -1

# Calculate returns, similar to your RSI strategy
hourly_nvda['returns'] = hourly_nvda['Adj Close'].pct_change()
hourly_nvda['strategy_returns'] = hourly_nvda['signal'].shift(1) * hourly_nvda['returns']
hourly_nvda['cumulative_strategy_returns'] = (1 + hourly_nvda['strategy_returns']).cumprod() - 1

# Initial capital to start with
initial_capital = 100

# Calculate portfolio value over time
hourly_nvda['portfolio_value'] = (1 + hourly_nvda['strategy_returns']).cumprod() * initial_capital

# Print performance summary
num_trades = hourly_nvda['signal'].abs().sum()
final_capital = hourly_nvda['portfolio_value'].iloc[-1]
total_return = (final_capital - initial_capital) / initial_capital * 100

print(f"Number of trades: {num_trades}")
print(f"Initial capital: ${initial_capital}")
print(f"Final capital: ${final_capital:.2f}")
print(f"Total return: {total_return:.2f}%")

In [ ]:
#Bollinger Bands Stratergy
period = 20
std_dev_multiplier = 2

# Calculate Simple Moving Average (SMA)
hourly_nvda['SMA'] = hourly_nvda['Adj Close'].rolling(window=period).mean()

# Calculate Standard Deviation
hourly_nvda['std_dev'] = hourly_nvda['Adj Close'].rolling(window=period).std()

# Calculate Upper and Lower Bollinger Bands
hourly_nvda['upper_band'] = hourly_nvda['SMA'] + (hourly_nvda['std_dev'] * std_dev_multiplier)
hourly_nvda['lower_band'] = hourly_nvda['SMA'] - (hourly_nvda['std_dev'] * std_dev_multiplier)

# Generate signals: buy when the price crosses above the lower band, sell when below the upper band
hourly_nvda['signal'] = 0
hourly_nvda.loc[hourly_nvda['Adj Close'] < hourly_nvda['lower_band'], 'signal'] = 1  # Buy signal
hourly_nvda.loc[hourly_nvda['Adj Close'] > hourly_nvda['upper_band'], 'signal'] = -1 # Sell signal

# Calculating returns as before
hourly_nvda['returns'] = hourly_nvda['Adj Close'].pct_change()
hourly_nvda['strategy_returns'] = hourly_nvda['signal'].shift(1) * hourly_nvda['returns']
hourly_nvda['cumulative_strategy_returns'] = (1 + hourly_nvda['strategy_returns']).cumprod() - 1

initial_capital = 100
hourly_nvda['portfolio_value'] = (1 + hourly_nvda['strategy_returns']).cumprod() * initial_capital

# Calculate and print metrics
num_trades = hourly_nvda['signal'].abs().sum()
final_capital = hourly_nvda['portfolio_value'].iloc[-1]
total_return = (final_capital - initial_capital) / initial_capital * 100

print(f"Number of trades: {num_trades}")
print(f"Initial capital: ${initial_capital}")
print(f"Final capital: ${final_capital:.2f}")
print(f"Total return: {total_return:.2f}%")

In [ ]:
window = 20
num_std_dev = 2

# Calculate Middle Band
hourly_nvda['middle_band'] = hourly_nvda['Adj Close'].rolling(window=window).mean()

# Calculate Standard Deviation
std_dev = hourly_nvda['Adj Close'].rolling(window=window).std()

# Calculate Upper and Lower Bands
hourly_nvda['upper_band'] = hourly_nvda['middle_band'] + (std_dev * num_std_dev)
hourly_nvda['lower_band'] = hourly_nvda['middle_band'] - (std_dev * num_std_dev)

# Generate signals
hourly_nvda['signal'] = 0
hourly_nvda.loc[hourly_nvda['Adj Close'] < hourly_nvda['lower_band'], 'signal'] = 1  # Buy
hourly_nvda.loc[hourly_nvda['Adj Close'] > hourly_nvda['upper_band'], 'signal'] = -1  # Sell

# Calculate returns
hourly_nvda['returns'] = hourly_nvda['Adj Close'].pct_change()
hourly_nvda['strategy_returns'] = hourly_nvda['signal'].shift(1) * hourly_nvda['returns']

In [ ]:
# Plot the strategy
plt.figure(figsize=(14, 7))

plt.plot(hourly_nvda['Adj Close'], label='Close Price', alpha=0.5)
plt.plot(hourly_nvda['middle_band'], label='Middle Band (20-SMA)', linestyle='--')
plt.plot(hourly_nvda['upper_band'], label='Upper Band', linestyle='--')
plt.plot(hourly_nvda['lower_band'], label='Lower Band', linestyle='--')

# Highlight buy and sell signals
plt.plot(hourly_nvda.loc[hourly_nvda['signal'] == 1].index,
         hourly_nvda['lower_band'][hourly_nvda['signal'] == 1],
         '^', markersize=10, color='g', lw=0, label='Buy Signal')

plt.plot(hourly_nvda.loc[hourly_nvda['signal'] == -1].index,
         hourly_nvda['upper_band'][hourly_nvda['signal'] == -1],
         'v', markersize=10, color='r', lw=0, label='Sell Signal')

plt.title('NVDA Hourly Close Price and Bollinger Bands')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
buy_signals = hourly_nvda[hourly_nvda['signal'] == 1]
sell_signals = hourly_nvda[hourly_nvda['signal'] == -1]

In [ ]:
fig = go.Figure()

# Add Close Price trace
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['Adj Close'], mode='lines', name='Close Price'))

# Add Bollinger Bands traces
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['upper_band'], line=dict(color='rgba(250, 0, 0, 0.75)'), name='Upper Band'))
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['middle_band'], line=dict(color='rgba(0, 0, 250, 0.75)'), name='Middle Band'))
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['lower_band'], line=dict(color='rgba(250, 0, 0, 0.75)'), name='Lower Band', fill='tonexty', fillcolor='rgba(250, 0, 0, 0.2)'))

# Add Buy signals trace
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Adj Close'], mode='markers', marker=dict(color='Green', size=10, symbol='triangle-up'), name='Buy Signal'))

# Add Sell signals trace
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Adj Close'], mode='markers', marker=dict(color='Red', size=10, symbol='triangle-down'), name='Sell Signal'))

# Update layout
fig.update_layout(title='NVDA Hourly Close Price and Bollinger Bands', xaxis_title='Date', yaxis_title='Price', template='plotly_dark')

fig.show()
fig.write_html("nvda_bb.html")

In [ ]:
def calculate_supertrend(df, period=7, multiplier=3):
    hl2 = (df['High'] + df['Low']) / 2
    atr = 'ATR_' + str(period)
    df[atr] = hl2.rolling(window=period).mean()
    df[atr] = df[atr].fillna(method='bfill')

    st = 'SuperTrend_' + str(period) + '_' + str(multiplier)
    df[st] = 0.0

    for i in range(1, len(df)):
        if df['Close'][i] > df[st][i - 1]:
            df[st][i] = hl2[i] + (multiplier * df[atr][i])
        else:
            df[st][i] = hl2[i] - (multiplier * df[atr][i])

    df[st] = df[st].shift(1)
    return df

# Assuming 'hourly_nvda' is your DataFrame
hourly_nvda = calculate_supertrend(hourly_nvda)

# Generating signals
hourly_nvda['signal'] = 0
hourly_nvda.loc[hourly_nvda['Close'] > hourly_nvda['SuperTrend_7_3'], 'signal'] = 1
hourly_nvda.loc[hourly_nvda['Close'] < hourly_nvda['SuperTrend_7_3'], 'signal'] = -1

# Assuming you want to calculate returns similar to your RSI strategy:
hourly_nvda['returns'] = hourly_nvda['Close'].pct_change()
hourly_nvda['strategy_returns'] = hourly_nvda['signal'].shift(1) * hourly_nvda['returns']
hourly_nvda['cumulative_strategy_returns'] = (1 + hourly_nvda['strategy_returns']).cumprod() - 1

initial_capital = 100
hourly_nvda['portfolio_value'] = (1 + hourly_nvda['strategy_returns']).cumprod() * initial_capital

# Calculate and print metrics
num_trades = hourly_nvda['signal'].abs().sum()
final_capital = hourly_nvda['portfolio_value'].iloc[-1]
total_return = (final_capital - initial_capital) / initial_capital * 100

print(f"Number of trades: {num_trades}")
print(f"Initial capital: ${initial_capital}")
print(f"Final capital: ${final_capital:.2f}")
print(f"Total return: {total_return:.2f}%")

In [ ]:
n = 10  # Number of periods used to calculate momentum

# Calculate momentum
hourly_nvda['momentum'] = hourly_nvda['Adj Close'] - hourly_nvda['Adj Close'].shift(n)

# Generate signals: buy (1) when momentum is positive, sell (-1) when negative
hourly_nvda['signal'] = 0
hourly_nvda.loc[hourly_nvda['momentum'] > 0, 'signal'] = 1
hourly_nvda.loc[hourly_nvda['momentum'] < 0, 'signal'] = -1

# Calculate daily returns
hourly_nvda['returns'] = hourly_nvda['Adj Close'].pct_change()

# Apply the strategy
hourly_nvda['strategy_returns'] = hourly_nvda['signal'].shift(1) * hourly_nvda['returns']

# Calculate cumulative returns of the strategy
hourly_nvda['cumulative_strategy_returns'] = (1 + hourly_nvda['strategy_returns']).cumprod() - 1

# Setting the initial capital to $100
initial_capital = 100

# Calculating total portfolio value
hourly_nvda['portfolio_value'] = (1 + hourly_nvda['strategy_returns']).cumprod() * initial_capital

# Printing the number of trades, initial capital, and final capital
num_trades = hourly_nvda['signal'].abs().sum()
final_capital = hourly_nvda['portfolio_value'].iloc[-1]
total_return = (final_capital - initial_capital) / initial_capital * 100

print('\n')
print(f"Number of trades: {num_trades}")
print(f"Initial capital: ${initial_capital}")
print(f"Final capital: ${final_capital:.2f}")
print(f"Total return: {total_return:.2f}%")
print('\n')

In [ ]:
# Parameters for the Greedy Strategy
buy_threshold = 0.15  # 5% increase from previous day
sell_threshold = -0.05  # 5% decrease from peak value after buying

hourly_nvda['returns'] = hourly_nvda['Adj Close'].pct_change()
hourly_nvda['signal'] = 0
holding = False
peak = None

for i in range(1, len(hourly_nvda)):
    if not holding:
        # Buy if the stock increased by more than buy_threshold
        if hourly_nvda['returns'].iloc[i] > buy_threshold:
            hourly_nvda['signal'].iloc[i] = 1
            holding = True
            peak = hourly_nvda['Adj Close'].iloc[i]
    else:
        current_price = hourly_nvda['Adj Close'].iloc[i]
        if current_price > peak:
            peak = current_price
        # Sell if the stock decreases by sell_threshold from its peak
        if (current_price - peak) / peak < sell_threshold:
            hourly_nvda['signal'].iloc[i] = -1
            holding = False

# Applying the strategy
hourly_nvda['strategy_returns'] = hourly_nvda['signal'].shift(1) * hourly_nvda['returns']
hourly_nvda['cumulative_strategy_returns'] = (1 + hourly_nvda['strategy_returns']).cumprod() - 1

# Calculating performance
initial_capital = 100
hourly_nvda['portfolio_value'] = (1 + hourly_nvda['strategy_returns']).cumprod() * initial_capital
num_trades = hourly_nvda['signal'].abs().sum()
final_capital = hourly_nvda['portfolio_value'].iloc[-1]
total_return = (final_capital - initial_capital) / initial_capital * 100

print('\n')
print(f"Number of trades: {num_trades}")
print(f"Initial capital: ${initial_capital}")
print(f"Final capital: ${final_capital:.2f}")
print(f"Total return: {total_return:.2f}%")
print('\n')

In [ ]:
# Plot the adjusted close price
fig = go.Figure()
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['Adj Close'], mode='lines', name='Adj Close'))

# Highlight buy signals
buy_signals = hourly_nvda[hourly_nvda['signal'] == 1]
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Adj Close'], mode='markers', marker=dict(color='Green', size=10, symbol='triangle-up'), name='Buy'))

# Highlight sell signals
sell_signals = hourly_nvda[hourly_nvda['signal'] == -1]
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Adj Close'], mode='markers', marker=dict(color='Red', size=10, symbol='triangle-down'), name='Sell'))

# Optionally, add a trace for portfolio value if you want to visualize how the strategy's performance evolves over time
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['portfolio_value'], mode='lines', name='Portfolio Value'))

fig.update_layout(title='Greedy Strategy: NVDA Hourly Trading Signals', xaxis_title='Date', yaxis_title='Price', template='plotly_dark')
fig.show()
fig.write_html("nvda_greedy.html")

In [ ]:
hourly_nvda

In [ ]:
short_window = 50
long_window = 200

# Calculate the moving averages
hourly_nvda['short_mavg'] = hourly_nvda['Adj Close'].rolling(window=short_window, min_periods=1).mean()
hourly_nvda['long_mavg'] = hourly_nvda['Adj Close'].rolling(window=long_window, min_periods=1).mean()

# Generate signals
hourly_nvda['signal'] = 0
# Buy signal
hourly_nvda.loc[hourly_nvda['short_mavg'] > hourly_nvda['long_mavg'], 'signal'] = 1
# Sell signal
hourly_nvda.loc[hourly_nvda['short_mavg'] < hourly_nvda['long_mavg'], 'signal'] = -1

# Calculate daily returns
hourly_nvda['returns'] = hourly_nvda['Adj Close'].pct_change()

# Apply the strategy
hourly_nvda['strategy_returns'] = hourly_nvda['signal'].shift(1) * hourly_nvda['returns']

# Calculate cumulative returns of the strategy
hourly_nvda['cumulative_strategy_returns'] = (1 + hourly_nvda['strategy_returns']).cumprod() - 1

# Setting the initial capital to $100
initial_capital = 100

# Calculating total portfolio value
hourly_nvda['portfolio_value'] = (1 + hourly_nvda['strategy_returns']).cumprod() * initial_capital

# Printing the results
num_trades = hourly_nvda['signal'].abs().sum()
final_capital = hourly_nvda['portfolio_value'].iloc[-1]
total_return = (final_capital - initial_capital) / initial_capital * 100

print('\n')
print(f"Number of trades: {num_trades}")
print(f"Initial capital: ${initial_capital}")
print(f"Final capital: ${final_capital:.2f}")
print(f"Total return: {total_return:.2f}%")
print('\n')

In [ ]:

# Calculate moving averages
short_window = 20
long_window = 50
hourly_nvda['short_mavg'] = hourly_nvda['Adj Close'].rolling(window=short_window, min_periods=1).mean()
hourly_nvda['long_mavg'] = hourly_nvda['Adj Close'].rolling(window=long_window, min_periods=1).mean()

# Generate signals
hourly_nvda['signal'] = 0
hourly_nvda['signal'][short_window:] = np.where(hourly_nvda['short_mavg'][short_window:] > hourly_nvda['long_mavg'][short_window:], 1, 0)
hourly_nvda['positions'] = hourly_nvda['signal'].diff()

# Plotting
fig = go.Figure()

# Plot Adjusted Closing Price and Moving Averages
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['Adj Close'], mode='lines', name='Adj Close', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['short_mavg'], mode='lines', name=f'Short {short_window}-SMA', line=dict(color='red')))
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['long_mavg'], mode='lines', name=f'Long {long_window}-SMA', line=dict(color='green')))

# Highlight Buy signals
fig.add_trace(go.Scatter(x=hourly_nvda[hourly_nvda['positions'] == 1].index,
                         y=hourly_nvda['short_mavg'][hourly_nvda['positions'] == 1],
                         mode='markers', marker_symbol='triangle-up', marker_line_color="green", marker_color="green",
                         marker_line_width=2, marker_size=10, name='Buy Signal'))

# Highlight Sell signals
fig.add_trace(go.Scatter(x=hourly_nvda[hourly_nvda['positions'] == -1].index,
                         y=hourly_nvda['short_mavg'][hourly_nvda['positions'] == -1],
                         mode='markers', marker_symbol='triangle-down', marker_line_color="red", marker_color="red",
                         marker_line_width=2, marker_size=10, name='Sell Signal'))

# Update layout
fig.update_layout(title='NVDA Trend Following Strategy', xaxis_title='Date', yaxis_title='Price', template='plotly_dark')

fig.show()
fig.write_html("nvda_trend.html")


In [ ]:
breakout_window = 20

# Calculate the rolling max and min for the breakout window
hourly_nvda['20_day_high'] = hourly_nvda['Adj Close'].rolling(window=breakout_window).max()
hourly_nvda['20_day_low'] = hourly_nvda['Adj Close'].rolling(window=breakout_window).min()

# Generate signals
# Buy signal when the close price breaks above the 20-day high
hourly_nvda['signal'] = 0
hourly_nvda.loc[hourly_nvda['Adj Close'] > hourly_nvda['20_day_high'].shift(1), 'signal'] = 1

# Sell signal when the close price breaks below the 20-day low
hourly_nvda.loc[hourly_nvda['Adj Close'] < hourly_nvda['20_day_low'].shift(1), 'signal'] = -1

# Assuming you will close your position at the next signal, calculate the returns
hourly_nvda['returns'] = hourly_nvda['Adj Close'].pct_change()
hourly_nvda['strategy_returns'] = hourly_nvda['signal'].shift(1) * hourly_nvda['returns']
hourly_nvda['cumulative_strategy_returns'] = (1 + hourly_nvda['strategy_returns']).cumprod() - 1

# Calculate initial capital and final portfolio value
initial_capital = 100
hourly_nvda['portfolio_value'] = (1 + hourly_nvda['strategy_returns']).cumprod() * initial_capital
final_capital = hourly_nvda['portfolio_value'].iloc[-1]
total_return = (final_capital - initial_capital) / initial_capital * 100

# Print results
print(f"Number of trades: {hourly_nvda['signal'].abs().sum()}")
print(f"Initial capital: ${initial_capital}")
print(f"Final capital: ${final_capital:.2f}")
print(f"Total return: {total_return:.2f}%")

In [ ]:
import plotly.graph_objects as go

# Plot the adjusted close price
fig = go.Figure()
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['Adj Close'], mode='lines', name='Adj Close', line=dict(color='blue')))

# Highlight the 20-day high and low as the breakout levels
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['20_day_high'], mode='lines', name='20-Day High', line=dict(color='red', dash='dot')))
fig.add_trace(go.Scatter(x=hourly_nvda.index, y=hourly_nvda['20_day_low'], mode='lines', name='20-Day Low', line=dict(color='green', dash='dot')))

# Mark buy signals
buy_signals = hourly_nvda[hourly_nvda['signal'] == 1]
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Adj Close'], mode='markers', marker_symbol='triangle-up', marker_line_color="green", marker_color="green", marker_line_width=2, marker_size=10, name='Buy Signal'))

# Mark sell signals
sell_signals = hourly_nvda[hourly_nvda['signal'] == -1]
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Adj Close'], mode='markers', marker_symbol='triangle-down', marker_line_color="red", marker_color="red", marker_line_width=2, marker_size=10, name='Sell Signal'))

# Update layout
fig.update_layout(title='NVDA Breakout Strategy Visualization', xaxis_title='Date', yaxis_title='Price', template='plotly_dark')

fig.show()
fig.write_html('nvda_breakout.html')

In [ ]:
# Calculate moving averages for the trend-following component
short_window = 20
long_window = 50
hourly_nvda['short_mavg'] = hourly_nvda['Adj Close'].rolling(window=short_window, min_periods=1).mean()
hourly_nvda['long_mavg'] = hourly_nvda['Adj Close'].rolling(window=long_window, min_periods=1).mean()

# Identify the trend direction
hourly_nvda['trend'] = 0
hourly_nvda.loc[hourly_nvda['short_mavg'] > hourly_nvda['long_mavg'], 'trend'] = 1
hourly_nvda.loc[hourly_nvda['short_mavg'] < hourly_nvda['long_mavg'], 'trend'] = -1

# Calculate the 20-day high and low for the breakout component
hourly_nvda['20_day_high'] = hourly_nvda['Adj Close'].rolling(window=20).max()
hourly_nvda['20_day_low'] = hourly_nvda['Adj Close'].rolling(window=20).min()

# Initialize the signal column
hourly_nvda['signal'] = 0

# Generate combined signals
# Buy signal: If price breaks above 20-day high and the trend is positive
hourly_nvda.loc[(hourly_nvda['Adj Close'] > hourly_nvda['20_day_high'].shift(1)) & (hourly_nvda['trend'] == 1), 'signal'] = 1

# Sell signal: If price breaks below 20-day low and the trend is negative
hourly_nvda.loc[(hourly_nvda['Adj Close'] < hourly_nvda['20_day_low'].shift(1)) & (hourly_nvda['trend'] == -1), 'signal'] = -1

# Calculate strategy returns
hourly_nvda['returns'] = hourly_nvda['Adj Close'].pct_change()
hourly_nvda['strategy_returns'] = hourly_nvda['signal'].shift(1) * hourly_nvda['returns']
hourly_nvda['cumulative_strategy_returns'] = (1 + hourly_nvda['strategy_returns']).cumprod() - 1

# Calculate initial capital and final portfolio value
initial_capital = 100
hourly_nvda['portfolio_value'] = (1 + hourly_nvda['strategy_returns']).cumprod() * initial_capital
final_capital = hourly_nvda['portfolio_value'].iloc[-1]
total_return = (final_capital - initial_capital) / initial_capital * 100

# Print results
print(f"Number of trades: {hourly_nvda['signal'].abs().sum()}")
print(f"Initial capital: ${initial_capital}")
print(f"Final capital: ${final_capital:.2f}")
print(f"Total return: {total_return:.2f}%")